In [1]:
import numpy as np
import pandas as pd
import string
import re

In [2]:
with open('businessCard.txt', mode='r', encoding='utf', errors='ignore') as f:
    text = f.read()

In [3]:
data = list(map(lambda x: x.split('\t'), text.split('\n')))

In [4]:
df = pd.DataFrame(data[1:],columns=data[0])

In [5]:
df.head()

,id,text,tag
0,000.jpeg,,O
1,000.jpeg,.,O
2,000.jpeg,040-4852,B-PHONE
3,000.jpeg,"""8881,""",I-PHONE
4,000.jpeg,90309,B-PHONE


In [6]:
whitespace = string.whitespace
punctuation= "!#$%&\'()*+:;<=>?[\\]^`{|}~"
tablewhitespace = str.maketrans('','',whitespace)
tablepunctuation = str.maketrans('','',punctuation)
def cleanText(txt):
    text = str(txt)
#     text = text.lower()
    remvoewhitespace = text.translate(tablewhitespace)
    removepunctuation = remvoewhitespace.translate(tablepunctuation)
    return str(removepunctuation)

In [7]:
df["text"] = df['text'].apply(cleanText)

In [8]:
dataClean = df.query("text != '' ")
dataClean.dropna(inplace=True)

/tmp/ipykernel_6386/3379659824.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataClean.dropna(inplace=True)


In [9]:
dataClean

,id,text,tag
1,000.jpeg,.,O
2,000.jpeg,040-4852,B-PHONE
3,000.jpeg,"""8881,""",I-PHONE
4,000.jpeg,90309,B-PHONE
5,000.jpeg,52549,I-PHONE
...,...,...,...
10437,290.jpeg,p,O
10439,290.jpeg,SrmrriSurres,B-ORG
10441,290.jpeg,Richard,B-NAME
10442,290.jpeg,Pretorius,I-NAME


In [10]:
# convert data into spacy format

In [11]:
group = dataClean.groupby(by='id')

In [12]:
cards = group.groups.keys()

In [13]:
allcardsdata = []
for card in cards:
    cardata = []
    group_array = group.get_group(card)[['text', 'tag']].values
    content = ''
    annotation = {'entities': []}
    start = 0
    end = 0

    for text, label in group_array:
        text = str(text)
        string_length = len(text) + 1

        start = end
        end = string_length

        if label != 'O':
            annot = (start, end-1, label)
            annotation['entities'].append(annot)

        content = content + text + " "
        
    cardata = (content, annotation)
    allcardsdata.append(cardata)

In [14]:
allcardsdata

[('. 040-4852 "8881," 90309 52549 Fi /laurelsoverseaseducation @ LAURELS OVERSEAS EDUCATIONAL CONSULTANCY PVT. LTD. Sea U.K AUSTRALIA CANADA IRELAND www.laurelseducation.com info@laurelseducation.com ',
  {'entities': [(2, 8, 'B-PHONE'),
    (9, 7, 'I-PHONE'),
    (8, 5, 'B-PHONE'),
    (6, 5, 'I-PHONE'),
    (2, 7, 'B-ORG'),
    (8, 8, 'I-ORG'),
    (9, 11, 'I-ORG'),
    (12, 11, 'I-ORG'),
    (12, 4, 'I-ORG'),
    (5, 4, 'I-ORG'),
    (8, 24, 'B-WEB'),
    (25, 25, 'B-EMAIL')]}),
 ('john smith marketing manager web www.psdgraphics.com phone 123-456-7890 mail email@psdgraphics.com ',
  {'entities': [(0, 4, 'B-NAME'),
    (5, 5, 'I-NAME'),
    (6, 9, 'B-DES'),
    (10, 7, 'I-DES'),
    (4, 19, 'B-WEB'),
    (6, 12, 'B-PHONE'),
    (5, 21, 'B-EMAIL')]}),
 ('Sau 0 98489 24441 dy "08672," 224441 /ENKATESWAPA wie ',
  {'entities': [(2, 5, 'B-PHONE'), (6, 5, 'I-PHONE'), (7, 12, 'B-ORG')]}),
 ('Prasad @ "9,96,31,73,53,59,49,04,00,000" i Flex Design Album Design Visiting Cards DTP Works Telug

In [15]:
# remove missing values

In [16]:
card_data_df = pd.DataFrame(allcardsdata,columns=['text','labels'])
card_data_df['isNull'] = card_data_df['labels'].apply(lambda x: 'Null' if len(x['entities']) ==0 
                                                      else 'Clean')

In [17]:
card_data_df.query('isNull == "Null"')

,text,labels,isNull
27,ae Be 3,{'entities': []},Null
60,"Le ""95,"" Nungambakkam High ""Road,"" ""Nungambakk...",{'entities': []},Null


In [18]:
# Consilder only clean data

In [19]:
card_data_df.dropna(inplace=True)
clean_data = card_data_df.query('isNull == "Clean"')[['text','labels']]

In [20]:
allcardsdata = list(map(lambda x: tuple(x), clean_data.values.tolist()))

In [21]:
# split data into train and test data

In [22]:
import random

random.shuffle(allcardsdata)

In [23]:
len(allcardsdata)

265

In [24]:
TrainData = allcardsdata[:240]
TestData = allcardsdata[240:]

In [25]:
import pickle

In [26]:
pickle.dump(TrainData, open('./data/TrainData.pickle', mode='wb'))
pickle.dump(TestData, open('./data/TestData.pickle', mode='wb'))